In [1]:
!pip install pathway bokeh panel --quiet

from google.colab import files
uploaded = files.upload()




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Saving dataset.csv to dataset.csv
Saving Modified - modified.csv to Modified - modified.csv
Saving parking_stream_full.csv to parking_stream_full.csv


In [8]:
import pandas as pd

# Load full dataset (should include Latitude and Longitude)
df = pd.read_csv("/content/dataset.csv")

# Timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Feature Engineering
df["VehicleWeight"] = df["VehicleType"].map({"car": 1, "bike": 0.5, "truck": 1.5}).fillna(1)
df["TrafficScore"] = df["TrafficConditionNearby"].map({"Low": 1, "Moderate": 2, "High": 3}).fillna(1)
df["QueueNorm"] = pd.to_numeric(df["QueueLength"], errors="coerce").fillna(0)
df["QueueNorm"] = df["QueueNorm"] / df["QueueNorm"].max()
df["TrafficNorm"] = df["TrafficScore"] / df["TrafficScore"].max()
df["IsSpecialDay"] = df["IsSpecialDay"].fillna(0).astype(int)

# ✅ Include Latitude and Longitude
df_stream = df[[
    "Timestamp", "ID", "Occupancy", "Capacity",
    "QueueNorm", "TrafficNorm", "IsSpecialDay", "VehicleWeight",
    "Latitude", "Longitude"
]]

# Overwrite the file
df_stream.to_csv("parking_stream_full.csv", index=False)

# Confirm
print("✅ Rewritten CSV with columns:")
print(df_stream.columns.tolist())


✅ Rewritten CSV with columns:
['Timestamp', 'ID', 'Occupancy', 'Capacity', 'QueueNorm', 'TrafficNorm', 'IsSpecialDay', 'VehicleWeight', 'Latitude', 'Longitude']


In [3]:
import pathway as pw
import datetime

# Match schema to CSV column names
class ParkingSchema(pw.Schema):
    Timestamp: str
    ID: str
    Occupancy: int
    Capacity: int

# Replay CSV in real-time (simulating stream)
data = pw.demo.replay_csv("parking_stream_full.csv", schema=ParkingSchema, input_rate=100)

# Parse timestamp format
fmt = "%Y-%m-%d %H:%M:%S"

# Add usable time columns
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [9]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    ID: str
    Occupancy: int
    Capacity: int
    QueueNorm: float
    TrafficNorm: float
    IsSpecialDay: int
    VehicleWeight: float
    Latitude: float
    Longitude: float

data = pw.demo.replay_csv("parking_stream_full.csv", schema=ParkingSchema, input_rate=100)
pw.run()


Output()

In [5]:
# Model 1: Baseline Linear Pricing

import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day + "-" + pw.this.ID,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        lot = pw.reducers.max(pw.this.ID),
        occ = pw.reducers.max(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + 2 * (pw.this.occ / pw.this.cap)
    )
)


In [11]:
import pandas as pd
df_check = pd.read_csv("parking_stream_full.csv")
print(df_check.columns.tolist())


['Timestamp', 'ID', 'Occupancy', 'Capacity', 'QueueNorm', 'TrafficNorm', 'IsSpecialDay', 'VehicleWeight', 'Latitude', 'Longitude']


In [12]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),

    ID = data.ID,
    Occupancy = data.Occupancy,
    Capacity = data.Capacity,
    QueueNorm = data.QueueNorm,
    TrafficNorm = data.TrafficNorm,
    IsSpecialDay = data.IsSpecialDay,
    VehicleWeight = data.VehicleWeight,
    Latitude = data.Latitude,
    Longitude = data.Longitude
)


In [13]:
#Model 2: Demand-Based Pricing Block

# Step 1: Window and reduce
delta_window_model2 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day + "-" + pw.this.ID,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        lot = pw.reducers.max(pw.this.ID),
        occ = pw.reducers.max(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity),
        queue = pw.reducers.max(pw.this.QueueNorm),
        traffic = pw.reducers.max(pw.this.TrafficNorm),
        special = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight = pw.reducers.max(pw.this.VehicleWeight)
    )
)

# Step 2: Compute demand
window_with_demand = delta_window_model2.with_columns(
    demand = (
        2 * (pw.this.occ / pw.this.cap) +
        1 * pw.this.queue -
        1 * pw.this.traffic +
        1 * pw.this.special +
        1 * pw.this.vehicle_weight
    )
)

# Step 3a: Normalize demand
window_with_norm = window_with_demand.with_columns(
    norm_demand = pw.this.demand / 5
)

# Step 3b: Calculate price (now norm_demand exists)
final_model2 = window_with_norm.with_columns(
    price = 10 + 5 * pw.this.norm_demand
)

final_model2 = final_model2.with_columns(
    day = pw.this.t.dt.strftime("%Y-%m-%dT00:00:00")
)





In [18]:
#Model 1
import bokeh.plotting
import panel as pn
pn.extension()

# Plot pricing for each time window
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Real-Time Parking Lot Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")



# Display the visualization
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [19]:
pw.run()


Output()

In [20]:
# Model 2
import bokeh.plotting
import panel as pn
pn.extension()

# Plot pricing for each time window
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 2: Demand-Based Pricing Block",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = final_model2.plot(price_plotter, sorting_col="t")


# Display the visualization
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [21]:
pw.run()

Output()